In [0]:
%sql
-- Célula 1 (SQL)
CREATE OR REPLACE TEMP VIEW bronze_fx AS
SELECT *
FROM delta.`abfss://bronze@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`;


In [0]:
%sql
-- Célula 2 (SQL)
SELECT base, symbol, rate, ref_date, provider, ingestion_ts
FROM bronze_fx
ORDER BY ingestion_ts DESC
LIMIT 20;


In [0]:
%sql
-- Célula 3 (SQL)
CREATE OR REPLACE TEMP VIEW silver_fx_stage AS
SELECT
  base,
  symbol,
  CAST(rate AS DOUBLE)          AS rate,
  TO_DATE(ref_date)             AS ref_date,
  COALESCE(provider,'unknown')  AS provider,
  ingestion_ts
FROM bronze_fx
WHERE symbol = 'BRL'
  AND base   = 'USD'
  AND ref_date IS NOT NULL
  AND rate IS NOT NULL
  AND CAST(rate AS DOUBLE) > 0;


In [0]:
%sql
-- Célula 4 (SQL)
CREATE OR REPLACE TEMP VIEW silver_fx_dedup AS
WITH ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY ref_date ORDER BY ingestion_ts DESC) AS rn
  FROM silver_fx_stage
)
SELECT base, symbol, rate, ref_date, provider, ingestion_ts
FROM ranked
WHERE rn = 1;


In [0]:
%sql
-- Célula 5 (SQL)
CREATE OR REPLACE TABLE delta.`abfss://silver@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`
USING delta
PARTITIONED BY (year, month)
AS
SELECT
  ref_date,
  base,
  symbol,
  rate,
  provider,
  ingestion_ts,
  YEAR(ref_date)  AS year,
  MONTH(ref_date) AS month
FROM silver_fx_dedup;


In [0]:
%sql
-- Célula 6 (SQL)
SELECT ref_date, base, symbol, rate, provider, ingestion_ts
FROM delta.`abfss://silver@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`
ORDER BY ref_date DESC;
